In [18]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib as mpl

from Functions import DateTime
from Functions import PdDataFeeder
from Functions import TradingEnv
from Functions import PygameRender
from Functions import MinMaxScaler
from Functions import simpleReward

In [17]:
df = pd.read_csv("/home/zymantas/Desktop/Training_data/1HData/ATOMUSDT1H.csv", index_col = 0)

DateTime(df)

,Open,High,Low,Close,Volume
datetime,,,,,
2023-02-10 17:00:00,13.735,13.895,13.570,13.770,384660.9
2023-02-10 18:00:00,13.770,13.845,13.580,13.605,130652.0
2023-02-10 19:00:00,13.605,13.745,13.535,13.740,106459.7
2023-02-10 20:00:00,13.740,13.830,13.705,13.785,93636.9
2023-02-10 21:00:00,13.785,13.920,13.730,13.880,144567.0
...,...,...,...,...,...
2024-04-02 05:00:00,11.535,11.541,11.011,11.293,932590.6
2024-04-02 06:00:00,11.293,11.344,11.226,11.266,159250.6
2024-04-02 07:00:00,11.266,11.312,11.195,11.219,151649.0


In [ ]:
bt = Backtest(df, TradingAlgo)

bt.run()
bt.optimize()
bt.plot()